# Modular Bending Actuator

## Installation
The dataset is available for this example and can be downloaded from  
Schindler and de Payrebrune [1] by CC BY. Copy the contents of the  
`data.zip` inside this folder, such that  a `calibration` and a  
`dataset_02` folder exist inside this directory.  

Since this example requires some modifications to the source base of  
icpReconstructor. It is recommended to generate a fresh virtual  
environment using  
```bash
python -m venv .venv
```  
in the main directory of the icpReconstructor repository. After  
activation of the virtual environment using  
```bash
source .venv/bin/activate
```
the modified local icpReconstructor can be installed using  
```bash
pip install -e .
``` 
If the install fails due to the line `version='{{VERSION_PLACEHOLDER}}'`  
in `setup.py`, replace it with `version='0.0.1'`.

This example requires additionally the installation of OpenCV and   
matplotlib. This can be done via  
```bash
pip install opencv-python matplotlib
```

## Code Sandbox

First, we import the necessary dependencies for this example. Some   
methods which are required by this example have been placed in the  
`utils.py` file for readability of this file. Tinkering with these  
methods is still possible, but not necessary.

In [ ]:
from pathlib import Path

import torch
import matplotlib.pyplot as pyplot

from icpReconstructor import TorchCurveEstimator

from icpReconstructor.extensions import (
    Module,
    ModularBendingActuator)

from utils import (
    load_calibration_parameters,
    load_images,
    preprocess_images,
    compute_projected_points)

### Global settings

The following settings are global and should only be changed with  
caution.

In [ ]:
# Paths
DATA_DIRECTORY = Path("./dataset_02/raw")
CALIBRATION_DIRECTORY = Path("./calibration")

# Number of cameras in the system
NUMBER_OF_CAMERAS = 2
# Schedule entry index (from dataset)
SCHEDULE_ENTRY_INDEX = 39

# Thresholds used for the binary images
THRESHOLD_MINIMUM_R = 200.0 / 255.0
THRESHOLD_MAXIMUM_G = 190.0 / 255.0
THRESHOLD_MAXIMUM_B = 140.0 / 255.0

# Force the usage of the CPU
USE_CPU = False

### Setup torch device

If a CUDA-Device is available, we will use this device unless explicitly  
prohibit by the `USE_CPU` flag above.

In [ ]:
# Device
device = torch.device(
    "cuda:0" if torch.cuda.is_available() and not USE_CPU else "cpu")
torch.set_default_device(device)
# Print default device
display(f"Using device: {torch.get_default_device()}")

### Load camera calibration data

Load the camera calibration parameters from the `calibration` folder.  
The calibration data needed in this example contains the camera matrix,  
projection matrix and distortion coefficients.

In [ ]:
# Load the calibration parameters of the cameras
calibration_parameters = load_calibration_parameters(
    CALIBRATION_DIRECTORY,
    NUMBER_OF_CAMERAS)

### Load images

In the following snippet, we load the images from the dataset and    
preprocess them to binary images using a color value threshold. For  
a different entry changed the `SCHEDULE_ENTRY_INDEX` in the global  
settings above. Note that this operation can take some time.

In [ ]:
# Filenames for target images
image_filenames = []
for camera_index in range(NUMBER_OF_CAMERAS):
    image_filename = str(
        Path(DATA_DIRECTORY) / 
        f"CROPPED_C{camera_index + 1}_E{SCHEDULE_ENTRY_INDEX}.png")

    image_filenames.append(image_filename)
        
# Load images for annotation
raw_images = load_images(image_filenames)

# Preprocess images with the given color value thresholds
target_images, target_image_indices = preprocess_images(
    raw_images,
    THRESHOLD_MINIMUM_R,
    THRESHOLD_MAXIMUM_G,
    THRESHOLD_MAXIMUM_B)

Plot the raw images and the binarized images with the thresholds from   
the global settings.

In [ ]:
# Plot 
figure = pyplot.figure()
gridspec = figure.add_gridspec(2, 2)
ax00 = figure.add_subplot(gridspec[0, 0])
ax01 = figure.add_subplot(gridspec[0, 1])
ax10 = figure.add_subplot(gridspec[1, 0])
ax11 = figure.add_subplot(gridspec[1, 1])
axes = [ax00, ax01, ax10, ax11]

ax00.imshow(raw_images[0])
ax01.imshow(raw_images[1])
ax10.imshow(target_images[0])
ax11.imshow(target_images[1])

ax00.set_title("First Camera (Raw)")
ax01.set_title("Second Camera (Raw)")
ax10.set_title("First Camera (Binarized)")
ax11.set_title("Second Camera (Binarized)")

for axis in axes:
    axis.set_axis_off()

### Backbone reconstruction

In the following snippet the reconstruction of the backbone is performed.  
The reconstruction is based on the method described in Hoffmann et al. [2]  
and on the modifications described in Schindler and de Payrebrune [3].

In [ ]:
# Parameters to play around with
# Slender or volumetric model
slender = False
# Curvature degree
curvature_degree = 3
# Weight factor between pixel and backbone loss
w = 0.1
# Number of estimate points along backbone for each actuator
number_of_steps: int = 10
# Learning rates 
learning_rates = (
    1.0,  # ... for the curvature coefficients
    0.01, # ... for the elongation coefficients
    0.01) # ... for the base position
# Distance norm for the loss function
distance_norm = 2
# Number of epochs
epochs = 40
# Batch size in pixels
batch_size = None

Create a bending actuator with a single module. We set the initial base  
position to a good guess and the base rotation to 150° as it is fixed by  
the base plate.

In [ ]:
# Bending Actuator with a single module using default parameters
module = Module()
modules = [ module ]

# Initial guess for the base position and rotation
base_position = torch.tensor([0.026, 0.092, 0.37])
base_rotation = torch.tensor(150) # degrees

# Create the bending actuator
bending_actuator = ModularBendingActuator(
    modules,
    base_position=base_position,
    base_rotation=base_rotation,
    slender=slender,
    curvature_degree=curvature_degree)

Create the curve estimator.

In [ ]:
# Length of the bending actuator
L = bending_actuator.l.sum()

# Curve Estimator
estimator = TorchCurveEstimator(
    bending_actuator,
    calibration_parameters, 
    l=L,
    w=w, 
    n_steps=number_of_steps, 
    dist_norm=distance_norm)


Create the optimizer and the learning rate scheduler.

In [ ]:
# Optimizer
learning_rates = learning_rates
parameter_groups = [
    { "params": [], "lr": lr } for lr in learning_rates ]
parameter_map = {
    "ux.u_p": 0,
    "uy.u_p": 0,
    "la.u_p": 1,
    "base_position": 2,
}

for name, parameter in bending_actuator.named_parameters():
    parameter_group_index = parameter_map.get(name, 0)
    parameter_groups[parameter_group_index]["params"].append(parameter)

optimizer = torch.optim.Adam(
    parameter_groups, 
    learning_rates[1], 
    weight_decay=0)

# Learning Rate Scheduler
lr_scheduler = torch.optim.lr_scheduler.LinearLR(
    optimizer=optimizer,
    start_factor=1.0,
    end_factor=0.5,
    total_iters=epochs)

Fit the backbone.

In [ ]:
figure = pyplot.figure(figsize=(4, 4))
display_handle = display("", display_id=True)

gridspec = figure.add_gridspec(1, 1)
axis_loss = figure.add_subplot(gridspec[0, 0])

def update(_):
    axis_loss.clear()
    axis_loss.grid()
    axis_loss.set_title("Loss")
    axis_loss.set_xlabel("Epoch")
    axis_loss.set_ylabel("Loss")
    axis_loss.set_xlim(0, epochs)

    axis_loss.plot(estimator.loss_history, color="blue")
    
    # Update the already printed figure
    display_handle.update(figure)

# Set callback for the post-epoch
estimator.post_epoch_cb = [update]

# Fit the backbone
# NOTE: We need to explicitly set the device here since the numpy to
#       torch conversion does not follow the set default device.
loss_history = estimator.fit(
    pixel_list=target_image_indices, 
    optimizer=optimizer, 
    scheduler=lr_scheduler,
    n_iter=epochs, 
    batch_size=batch_size,
    device=torch.get_default_device())

## Plot the reconstructed backbone
Finally we plot the reconstructed backbone as blue dashed line and the  
estimate points as white markers.

In [ ]:
# Length discretization
s_values = estimator.s_val

# Solve for the backbone
p_values, R_values = bending_actuator.solve_backbone(s_values)

# Solve for the chamber backbones
estimate_points = bending_actuator.generate_estimate_points(
    s_values, p_values, R_values)

# Detach estimate points points for further computation in the 
# visualization stage
estimate_points = estimate_points.detach()

# Backbone
backbone = bending_actuator.from_actuator_space_to_world_space(p_values)
backbone = backbone.detach()

# Project basepoint to images
#  Detach 
base_position = bending_actuator.base_position.detach()
#  Unsqueeze because we need the batched form for points
base_position = base_position.unsqueeze(0)

# Convert the base position to image space to see if it fits the image
projected_base_position = compute_projected_points(
    base_position, calibration_parameters).squeeze(1)

# Project estimate points to images
projected_estimate_points = compute_projected_points(
    estimate_points, calibration_parameters)

# Project bacbkone to images
projected_backbone = compute_projected_points(
    backbone, calibration_parameters)

for camera_index, image in enumerate(raw_images):
    # Create figure
    figure = pyplot.figure()
    axis = figure.add_subplot()

    axis.imshow(image, aspect="auto")
    axis.set_axis_off()

    # Plot settings
    scatter_settings = {
        "marker": "P",
        "edgecolors": "black"
    }

    axis.plot(
        projected_backbone[camera_index, :, 0],
        projected_backbone[camera_index, :, 1],
        linestyle="--",
        color="blue",
        label="Backbone")

    axis.scatter(
        projected_base_position[camera_index, 0], 
        projected_base_position[camera_index, 1], 
        label="Base Position", c="blue", **scatter_settings)

    axis.scatter(
        projected_estimate_points[camera_index, :, 0],
        projected_estimate_points[camera_index, :, 1],
        label="Estimate Points", c="white", **scatter_settings)

## Acknowledgement

Funded by the Deutsche Forschungsgemeinschaft (DFG, German Research   
Foundation) – 501861263 – SPP2353

## References
[1] Schindler, L. and de Payrebrune, K.: Data for "Image-based Backbone  
Reconstruction for Non-Slender Soft Robots". Zenodo, 2024.  
DOI: 10.5281/zenodo.11352738

[2] M. K. Hoffmann, J. Mühlenhoff, Z. Ding, T. Sattel and K. Flaßkamp.  
An iterative closest point algorithm for marker-free 3D shape  
registration of continuum robots. arXiv preprint, (2024).   
DOI: 10.48550/arXiv.2405.15336

[3] Schindler, L. and de Payrebrune, K.: Image-based backbone  
reconstruction for non-slender soft robots. Proceedings in Applied  
Mathematics, 2024. (Submitted)